# Lab 3: Policy Search

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The player **taking the last object wins**.

* Task3.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task3.2: An agent using evolved rules
* Task3.3: An agent using minmax
* Task3.4: An agent using reinforcement learning

## Instructions

* Create the directory `lab3` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

## Deadlines ([AoE](https://en.wikipedia.org/wiki/Anywhere_on_Earth))

* Sunday, December 4th for Task3.1 and Task3.2
* Sunday, December 11th for Task3.3 and Task3.4
* Sunday, December 18th for all reviews

In [216]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor
from functools import reduce

## The *Nim* and *Nimply* classes

In [217]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [218]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [219]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["valide_row"]= [ r for r,c in enumerate(state.rows) if state._rows[r]>0]
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

## Strategies

In [220]:
def pure_random(state: Nim) -> Nimply:
    '''simple strategy, it just choose randomly from the rows'''
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))
       
def optimal_startegy(state: Nim) -> Nimply:
    '''optimal_startegy is based on brute force approach ''' 
    data = cook_status(state)
    # with next we give back the moves once the object to remove has been removed
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

def evolve(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        '''evolvable strategy consists in a choice between the shortest row or he longest one'''
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable


def safeness(state: Nim):
    return reduce((lambda x,y: x ^ y), state._rows)

def comeback(state: Nim):  
    actual_cost = 10000
    *_, XOR = accumulate(state.rows, xor)
    r_cost = 0
    for _, r in enumerate(state._rows):
        #print(r)
        new_cost = abs((XOR ^ r) - r)
        # search for a minimum cost from the possible moves
        if (actual_cost > new_cost):
            r_cost = ((XOR ^ r) - r)
            if r_cost < 0: 
                actual_cost = (XOR ^ r) - r
            idx = r
    # returns the r_cost found that is the minimum for the possible moves
    return state._rows.index(idx),-r_cost

def expert_system_wikipedia(state: Nim):
    # An agent using fixed rules based on *nim-sum* by the rules of safeness described on wikipedia
    # https://it.wikipedia.org/wiki/Nim
    
    safe = reduce((lambda x,y: x ^ y), state._rows) == 0 # a safe state 
    
    if all([n <= 1 for n in state._rows]) or safe:  # if the rows have only 1s and 0s value, there is no strategy, we proceed randomly
        return pure_random(state)
    else: #not a safe state -> look for safety, the player looks to leave the board in a safe state
        return comeback(state)

In [221]:
NUM_MATCHES = 1000
NIM_SIZE = 10


def evaluate(strategy1: Callable, strategy2: Callable) -> float:
    '''evaluate put a first strategy versus a second one for a certain number of times, the results is the ratio 
    between the won matches and the total number of played matches'''
    opponent = (strategy1, strategy2)
    won = 0

    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        #player = 1
        player = random.randint(0,1) #make a random start
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
        if player == 1:
            won += 1
    return round(won / NUM_MATCHES, 2)

ev_optimalvsexpert = evaluate(optimal_startegy, expert_system_wikipedia)
logging.info(f"Expert vs optimal => {ev_optimalvsexpert}")
ev_randomvsexpert = evaluate(pure_random, expert_system_wikipedia)
logging.info(f"Expert vs random => {ev_randomvsexpert}")


ev_evolve05vsexpert = evaluate(evolve({'p':0.5}), expert_system_wikipedia)
logging.info(f"Expert vs evolve_strategy(p=0.5) => {ev_evolve05vsexpert}")


ev_optimalvsoptimal = evaluate(optimal_startegy, optimal_startegy)
logging.info(f"Optimal vs optimal => {ev_optimalvsoptimal}")


ev_optimalvsevolve05 = evaluate(optimal_startegy, evolve({'p':0.5}))
logging.info(f"Optimal vs evolve_strategy(p=0.5) => {ev_optimalvsevolve05}")

ev_purerandomvsevolve01 = evaluate(pure_random, evolve({'p':0.000001}))
logging.info(f"Pure_random vs evolve_strategy(p=0.1) => {ev_purerandomvsevolve01}")
ev_purerandomvsevolve05 = evaluate(pure_random, evolve({'p':0.5}))
logging.info(f"Pure_random vs evolve_strategy(p=0.5) => {ev_purerandomvsevolve05}")

ev_purerandomvsevolve09 = evaluate(pure_random, evolve({'p':0.99999}))
logging.info(f"Pure_random vs evolve_strategy(p=0.9) => {ev_purerandomvsevolve09}")


INFO:root:Expert vs optimal => 0.51
INFO:root:Expert vs random => 0.0
INFO:root:Expert vs evolve_strategy(p=0.5) => 0.0
INFO:root:Optimal vs optimal => 0.46
INFO:root:Optimal vs evolve_strategy(p=0.5) => 1.0
INFO:root:Pure_random vs evolve_strategy(p=0.1) => 0.49
INFO:root:Pure_random vs evolve_strategy(p=0.5) => 0.5
INFO:root:Pure_random vs evolve_strategy(p=0.9) => 0.52


## Match

In [222]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (expert_system_wikipedia, optimal_startegy)

nim = Nim(6)
logging.debug(f"status: Initial board  -> {nim}")
player = random.randint(0,1)
#player = 0
while nim:  #while nim is > 0, i.e. we have at least an object to remove
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11>
DEBUG:root:status: After player 0 -> <1 1 5 7 9 11>
DEBUG:root:status: After player 1 -> <1 1 2 7 9 11>
DEBUG:root:status: After player 0 -> <1 1 2 0 9 11>
DEBUG:root:status: After player 1 -> <1 1 2 0 6 11>
DEBUG:root:status: After player 0 -> <1 1 2 0 6 4>
DEBUG:root:status: After player 1 -> <1 1 2 0 6 3>
DEBUG:root:status: After player 0 -> <1 1 2 0 1 3>
DEBUG:root:status: After player 1 -> <1 1 2 0 1 0>
DEBUG:root:status: After player 0 -> <1 1 1 0 1 0>
DEBUG:root:status: After player 1 -> <0 1 1 0 1 0>
DEBUG:root:status: After player 0 -> <0 1 0 0 1 0>
DEBUG:root:status: After player 1 -> <0 1 0 0 0 0>
DEBUG:root:status: After player 0 -> <0 0 0 0 0 0>
INFO:root:status: Player 0 won!


# Task 3.2 Evolving Rules

## Evaluate GA algorithm function

In [223]:
#EVALUATION FUNCTION

NUM_MATCHES = 20
NIM_SIZE = 10
EVALUATE=pure_random


def evaluate_GA(strategy: Callable) -> float:
    
    opponent = (strategy, EVALUATE)
    won = 0
    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 0
        INDEX = 0
        while nim:
            if(player==0):
                ply = opponent[player](nim,INDEX)
                INDEX+=1
            else:
                ply = opponent[player](nim)
            nim.nimming(ply)
            player = 1 - player
            
        if player == 1:
            won += 1
    return won / NUM_MATCHES

## Possible Policy

In [224]:
LAST_CONDITION=False

def take_first(state: Nim, el: str):
    
    data = cook_status(state)
    
    if(LAST_CONDITION==True):
        if len(data["valide_row"])==1:
            #Victory Condition
            col=data["valide_row"][0]
            el=state._rows[col]
            return (col,el)

    col=data["valide_row"][0]
    n=state._rows[col]

    if el=="take_one":
        return (col,1)
    elif el=="take_all":
        return (col,n)
    elif el=="take_all_except_one":
        if n>1:
            return (col,n-1)
        else:
            return (col,1)
    else:
        return (col,1)

def take_last(state: Nim, el: str):
   
    data = cook_status(state)
   
    if(LAST_CONDITION==True):
        if len(data["valide_row"])==1:
                #Victory Condition
                col=data["valide_row"][0]
                el=state._rows[col]
                return (col,el)
    
   
    col=data["valide_row"][-1]
    n=state._rows[col]
    
    if el=="take_one":
        return (col,1)
    elif el=="take_all":
        return (col,n)
    elif el=="take_all_except_one":
        if n>1:
            return (col,n-1)
        else:
            return (col,1)
    else:
        return (col,1)

def take_first_odd(state: Nim, el: str):

    data = cook_status(state)
    
    if(LAST_CONDITION==True):
        if len(data["valide_row"])==1:
            #Victory Condition
            col=data["valide_row"][0]
            el=state._rows[col]
            return (col,el)



    col=data["valide_row"]
    Find=False
    for i in col:
        n=state._rows[i]
        if (n % 2) != 0:
            my_col=i
            Find=True
            break
    
    #NO ODD->Take first
    if(Find==False):
        my_col=data["valide_row"][0]


    n=state._rows[my_col]

    if el=="take_one":
        return (my_col,1)
    elif el=="take_all":
        return (my_col,n)
    elif el=="take_all_except_one":
        if n>1:
            return (my_col,n-1)
        else:
            return (my_col,1)
    else:
        return (my_col,1)


def take_first_even(state: Nim, el: str):
    
    data = cook_status(state)

    if(LAST_CONDITION==True):
        if len(data["valide_row"])==1:
            #Victory Condition
            col=data["valide_row"][0]
            el=state._rows[col]
            return (col,el)



    col=data["valide_row"]
    Find=False
    for i in col:
        n=state._rows[i]
        if (n % 2 and n>0) == 0:
            my_col=i
            Find=True
            break
    
    #NO even->Take first
    if(Find==False):
        my_col=data["valide_row"][0]

    n=state._rows[my_col]

    if el=="take_one":
        return (my_col,1)
    elif el=="take_all":
        return (my_col,n)
    elif el=="take_all_except_one":
        if n>1:
            return (my_col,n-1)
        else:
            return (my_col,1)
    else:
        return (my_col,1)

def take_longest(state: Nim, el: str):
    
    data = cook_status(state)
    if(LAST_CONDITION==True):
        if len(data["valide_row"])==1:
            #Victory Condition
            col=data["valide_row"][0]
            el=state._rows[col]
            return (col,el)


    col=data["longest_row"]
    n=state._rows[col]

    if el=="take_one":
        return (col,1)
    elif el=="take_all":
        return (col,n)
    elif el=="take_all_except_one":
        if n>1:
            return (col,n-1)
        else:
            return (col,1)
    else:
        return (col,1)

def take_shortest(state: Nim, el: str):
    
    data = cook_status(state)
    if(LAST_CONDITION==True):
        if len(data["valide_row"])==1:
            #Victory Condition
            col=data["valide_row"][0]
            el=state._rows[col]
            return (col,el)

    col = data["shortest_row"]
    n=state._rows[col]

    #Switch that not exist in python >.>
    if el=="take_one":
        return (col,1)
    elif el=="take_all":
        return (col,n)
    elif el=="take_all_except_one":
        if n>1:
            return (col,n-1)
        else:
            return (col,1)
    else:
        return (col,1)

def possible_policy(Row_choice,Element_choice):   
    POSSIBLE_POLICY=[]
    for i in Row_choice:
        for j in Element_choice:
            POSSIBLE_POLICY.append((i,j))
    return POSSIBLE_POLICY



def fitness(policy: list) -> float:
    return evaluate_GA(make_strategy_GA(policy))


def make_strategy_GA(policy: list)->Callable:

    def strategy(state:Nim,INDEX)->Nimply:
        N=len(policy)
        P=policy
        if INDEX<N:
            fun=P[INDEX]
        else:
            fun=P[INDEX%N]

        c,el=fun[0](state,fun[1])
        return Nimply(c,el)

    return strategy



## GA functions

In [225]:
NUM_GENERATIONS=6
POPULATION_SIZE=20
OFFSPRING_SIZE=20
TOURNAMENT_SIZE=4

population=[]


def GA_algorithm():

    Row_choice=[take_first,take_last,take_first_odd,take_first_even,take_shortest,take_longest]

    Element_choice=["take_all","take_one","take_all_except_one"]

    population = list()
    
    policies = possible_policy(Row_choice,Element_choice)
    
    
    for genome in policies:
        population.append([genome])
    while len(population)<POPULATION_SIZE:
        population.append([random.choice(policies)])

    Evolution(population,policies)
    


def Evolution(population, generator):
    '''Probability to get a crossover is bigger than a mutation'''
    for g in range(NUM_GENERATIONS):
        offspring = list()
        for i in range(OFFSPRING_SIZE):
            if random.random() < 0.5:
                p = tournament(population)
                o = mutation(p,generator)
            else:
                p1 = tournament(population)
                p2 = tournament(population)
                o = cross_over(p1, p2)
            
            offspring.append(o)
        population += offspring
        population = sorted(population, key=lambda i: fitness(i), reverse=True)[:POPULATION_SIZE]

    logging.info("\nTop five performance: ")


    performance=list()
    for i in population:
        performance.append(fitness(i)*100)

    for i in sorted(performance,key= lambda i : i)[0:5]:
        logging.info(f"{i}%")    



def tournament(population, tournament_size=TOURNAMENT_SIZE):
    choice=random.choices(population, k=tournament_size)
    if len(choice[0])<5:
        return choice[0]
    else:
        return max(choice, key=lambda i: fitness(i))




def cross_over(g1, g2):
    if(len(g1)==1 or len(g2)==1):

        return g1.__add__(g2)
        
    cut = random.randint(0, len(g1))
    c1=g2[:cut]+g1[cut:]
    c2=g1[:cut] + g2[cut:]
    return c1


def mutation(g,generator):
    '''This function has equal probability to remove or add a gene to the Individual only if it has more than one gene'''
    #remove element
    if random.random()>0.5 and len(g)>1:
        g.pop(random.randint(0,len(g)-1))

    #Add element
    else:
        for i in generator:
            m=random.choice(generator)
            if g.__contains__(m):
                continue
            else:
                g.append(m)
                break
    return g 

# EVALUATION

#### Evaluate against pure random with and without last_condition
#### Evaluate against professor's strategy with and without last_condition
#### Evaluate against optimal strategy

In [226]:


LAST_CONDITION=False
NUM_GENERATIONS=7
POPULATION_SIZE=30
OFFSPRING_SIZE=30
TOURNAMENT_SIZE=6

logging.info("TEST GENETIC ALGORITHM AGAINST:")


logging.info("\n\nPURE RANDOM, LAST CONDITION = FALSE")
EVALUATE=pure_random
GA_algorithm()


logging.info("\n\nPURE RANDOM, LAST CONDITION = TRUE")
LAST_CONDITION=True
EVALUATE=pure_random
GA_algorithm()



INFO:root:TEST GENETIC ALGORITHM AGAINST:
INFO:root:

PURE RANDOM, LAST CONDITION = FALSE
INFO:root:
Top five performance: 
INFO:root:50.0%
INFO:root:60.0%
INFO:root:65.0%
INFO:root:65.0%
INFO:root:70.0%
INFO:root:

PURE RANDOM, LAST CONDITION = TRUE
INFO:root:
Top five performance: 
INFO:root:65.0%
INFO:root:65.0%
INFO:root:75.0%
INFO:root:75.0%
INFO:root:75.0%


In [227]:


logging.info("\n\nPROFESSOR STRATEGY, LAST CONDITION = TRUE")

EVALUATE=evolve({'p':0.5})
LAST_CONDITION=False
GA_algorithm()


logging.info("\n\nPROFESSOR STRATEGY, LAST CONDITION = FALSE")

EVALUATE=evolve({'p':0.5})
LAST_CONDITION=True
GA_algorithm();


INFO:root:

PROFESSOR STRATEGY, LAST CONDITION = TRUE
INFO:root:
Top five performance: 
INFO:root:45.0%
INFO:root:45.0%
INFO:root:50.0%
INFO:root:55.00000000000001%
INFO:root:60.0%
INFO:root:

PROFESSOR STRATEGY, LAST CONDITION = FALSE
INFO:root:
Top five performance: 
INFO:root:70.0%
INFO:root:70.0%
INFO:root:75.0%
INFO:root:80.0%
INFO:root:80.0%


In [228]:


'''
NUM_GENERATIONS=10
POPULATION_SIZE=100
OFFSPRING_SIZE=100
TOURNAMENT_SIZE=10
'''

logging.info("\n\nOPTIMAL STRATEGY, LAST CONDITION = TRUE")
LAST_CONDITION=True
EVALUATE=expert_system_wikipedia
GA_algorithm()

INFO:root:

OPTIMAL STRATEGY, LAST CONDITION = TRUE
INFO:root:
Top five performance: 
INFO:root:0.0%
INFO:root:0.0%
INFO:root:0.0%
INFO:root:0.0%
INFO:root:0.0%
